In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from app.database import SessionLocal, engine, Base
from app.models import Fencer, Tournament, Bout
import datetime

In [3]:
Base.metadata.drop_all(engine)

In [4]:
Base.metadata.create_all(engine)

In [5]:
from web_scrapping_helpers import FIEPage, parse_metadata, parse_pool, pool_to_db, tournament_to_db

In [6]:
Scraper = FIEPage('epee', 's', 'A', 2)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\liamc\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [7]:
Scraper.open_competions().previous().age_group('s').comp_category('A').select_weapon('epee').select_gender('m').select_event_type()

In [9]:
metadata = Scraper.get_competitions()

In [10]:
comp = metadata[0]

In [11]:
with SessionLocal as session, session.begin():
    tourn_meta, link = comp
    tournament = tournament_to_db(tourn_meta, session)
    main = Scraper.open_window(link)
    pool_data = Scraper.to_results().to_pools().get_pools_data()
    for pool in pool_data:
        bouts = parse_pool(pool)
        pool_to_db(bouts, tournament, session)





In [ ]:
for fencer in test_cases:
    first_names = []
    last_names = []
    for name in fencer:
        if name.isupper():
            name = ''.join([name[0], name[1:].lower()])
            last_names.append(name)
        else: first_names.append(name)
    first_name = ' '.join(first_names)
    last_name = ' '.join(last_names)
    print(first_name, last_name)


In [15]:
# session = SessionLocal
# session.begin()
fencer_test = session.query(Fencer).filter_by(first_name = 'SAMUEL').all()

In [19]:
fencer_test[1].bout_lose

[PAUL ALLEGRE 5 to SAMUEL LARSEN 3,
 ALEXIS BAYARD 5 to SAMUEL LARSEN 0,
 MYEONGKI KIM 5 to SAMUEL LARSEN 3,
 YOHITO MASUDA 4 to SAMUEL LARSEN 3,
 JAKOB STYMNE 5 to SAMUEL LARSEN 2]

In [16]:
dc = {'a':[], 'b':[]}
len(dc)

2

In [15]:
win_bouts = fencer_test.bout_win
for bout in win_bouts:
    print(0)

0
0
0
0


[autoreload of app.models failed: Traceback (most recent call last):
  File "c:\Users\liamc\anaconda3\envs\fencing_elo\lib\site-packages\IPython\extensions\autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\liamc\anaconda3\envs\fencing_elo\lib\site-packages\IPython\extensions\autoreload.py", line 455, in superreload
    module = reload(module)
  File "c:\Users\liamc\anaconda3\envs\fencing_elo\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "c:\Users\liamc\Desktop\fencing_elo\app\models.py", line 13, in <module>
    tourn_fen = Table(
  File "<string>", line 2, in __new__
  File "c:\Users\liamc\anaconda3\envs\fencing_elo\lib\site-packages\sqlalchemy\util\deprecations.py", line 309, in warned


In [16]:
win_bouts

[autoreload of app.models failed: Traceback (most recent call last):
  File "c:\Users\liamc\anaconda3\envs\fencing_elo\lib\site-packages\IPython\extensions\autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\liamc\anaconda3\envs\fencing_elo\lib\site-packages\IPython\extensions\autoreload.py", line 455, in superreload
    module = reload(module)
  File "c:\Users\liamc\anaconda3\envs\fencing_elo\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "c:\Users\liamc\Desktop\fencing_elo\app\models.py", line 13, in <module>
    tourn_fen = Table(
  File "<string>", line 2, in __new__
  File "c:\Users\liamc\anaconda3\envs\fencing_elo\lib\site-packages\sqlalchemy\util\deprecations.py", line 309, in warned


NameError: name 'win_bouts' is not defined